In [1]:
#addprocs(4);
using DistributedArrays
using ForwardDiff
using StaticArrays
include("Stencil.jl")
include("stencilgmres.jl")
include("reservoirfunc.jl")
include("distributed.jl")
include("res3d.jl")
include("preconditioning.jl")
using BenchmarkTools

In [2]:
permdata = open("spe_perm.dat")
phidata  = open("spe_phi.dat")
raw1 = readdlm(phidata)
raw2 = readdlm(permdata)
close(permdata)
close(phidata)
porosity = reshape(raw1', 60, 220, 85)
lineperm = reshape(raw2', 3366000, 1)
kx = reshape(lineperm[1:1122000]', 60, 220, 85)
ky = reshape(lineperm[1122001:2244000]', 60, 220, 85)
kz = reshape(lineperm[2244001:end]', 60, 220, 85)
savepor = copy(porosity)
## Porosity pre-processing
for id in eachindex(porosity)
    if abs(porosity[id])<1e-3
        porosity[id] = 0.2
        kx[id], ky[id], kz[id] = 1e-10, 1e-10, 1e-10
    end
end
kraw = [SVector{3,Float64}([kx[i,j,k], ky[i,j,k], kz[i,j,k]]) for i in 1:60, j in 1:220, k in 1:85];

In [15]:
Nx = 50
Ny = 50
Nz = 50
offset = 1;

In [16]:
p_ref = 14.7 
C_r   = 3e-6
ϕ_ref = 0.2
#=everywhere=# ϕ     = porosity[offset:(Nx+offset-1), offset:(Ny+offset-1), offset:(Nz+offset-1)]
S_wc, S_or = 0.2, 0.2
k_r_w(x)   = ((x-S_wc)/(1-S_wc-S_or))^2
k_r_o(x)   = (1 - (x-S_wc)/(1-S_wc-S_or))^2
p_cow(x)   = 0
C_water    = 3e-6
C_oil      = 3e-6
ρ_water(p) = 64.0*exp(C_water*(p-p_ref)) 
ρ_oil(p)   = 53.0*exp(C_oil*(p-p_ref))   
μ_water = 0.3 # cp
μ_oil   = 3.0; # cp
saveper = (kx.+ky.+kz)./3
saveper = saveper[offset:(Nx+offset-1), offset:(Ny+offset-1), offset:(Nz+offset-1)];

In [5]:
## 3d model
Lx = 1200
Ly = 2200
Lz = 170
Δt = 0.01
Tf = 2000.0
Δx = (fill(Lx/Nx, Nx, Ny, Nz))
Δy = (fill(Ly/Ny, Nx, Ny, Nz))
Δz = fill(Lz/Nz, Nx, Ny, Nz)
z  = fill(12000.0, Nx, Ny, Nz)
k = makegrid(kraw[(offset):(Nx+offset-1), offset:(Ny+offset-1), offset:(Nz+offset-1)],7)
model = Reservoirmodel(Δt, Tf, (Δx, Δy, Δz), z, k, p_ref, C_r, ϕ_ref, ϕ, 
                k_r_w, k_r_o, p_cow, C_water, C_oil, ρ_water, ρ_oil, μ_water, μ_oil);

In [6]:
## Porosity proportional control
Total = 40.0*Nx
q = zeros(Nx, Ny, Nz, 2)
for i in (1,Nx), j in (1,Ny), k in 1:Nz
    q[i,j,k,2] = Total*(saveper[i,j,k]/sum(saveper[i,j,:]))/4
end
halfx, halfy = round(Int, Nx/2),round(Int, Ny/2)
for k in 1:Nz
    q[halfx,halfy,k,1] = -Total*(saveper[halfx,halfy,k]/sum(saveper[halfx,halfy,:]))
end 
#=## Constant rate well control
Total = 1000.0
q = zeros(Nx, Ny, Nz, 2)
for x in (1,Nx), y in (1,Ny), z in 1:Nz
    q[x, y, z, 1] = Total/4/Nz
end
for z in 1:Nz
    q[round(Int, Nx/2),round(Int, Ny/2),z,2] = -Total/Nz
end
q_d = distribute(q);=#

In [7]:
init       = [SVector{2,Float64}([6000.0,0.2]) for i in 1:Nx, j in 1:Ny, k in 1:Nz]
testgrid   = makegrid(init, 7);
g_guess   = testgrid;

In [8]:
solveprec1(model, q, testgrid, 2, printt=true, gmrestol=1e-8); #compiling purpose

day 1...


In [9]:
psps = solveprec1(model, q, testgrid, 10, printt=true, gmrestol=1e-8);

day 1...
day 2...
day 3...
day 4...
day 5...
day 6...
day 7...
day 8...
day 9...


In [11]:
using Plots

In [14]:
anim = @animate for t in 1:10
    heatmap([psps[t][i,j,1][1] for i in 1:Nx, j in 1:Ny])
end
gif(anim)

INFO: Saved animation to /Users/sw2030/Desktop/gitrep/Reservoirnew/src/tmp.gif


Plots.AnimatedGif("/Users/sw2030/Desktop/gitrep/Reservoirnew/src/tmp.gif")

In [40]:
S = getstencil(model, q, g_guess, testgrid)
res = getresidual(model, q, g_guess, testgrid);
copyres = getresidual(model, q, g_guess, testgrid);

In [46]:
tmp = Base.setindex(S[3,4,5].value,@SMatrix(zeros(2,2)),4)
tmp = Base.setindex(tmp,@SMatrix(zeros(2,2)),5)

([0.0 0.0; 0.730047 0.0], [0.0 0.0; 0.263764 0.0], [0.0 0.0; 0.00247713 0.0], [0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0], [0.0 0.0; 0.135921 0.0], [0.0 0.0; 0.523326 0.0])

In [47]:
S[3,4,5].value

([0.0 0.0; 0.730047 0.0], [0.0 0.0; 0.263764 0.0], [0.0 0.0; 0.00247713 0.0], [-0.370505 -6.17508e5; -2.88643 5.11374e5], [0.0 0.0; 0.00359579 0.0], [0.0 0.0; 0.135921 0.0], [0.0 0.0; 0.523326 0.0])

In [9]:
N = Nx*Ny*Nz
Nxy = Nx*Ny
A = spzeros(2N,2N)
A2= spzeros(StaticArrays.SArray{Tuple{2,2},Float64,2,4},N,N)
b = zeros(2N)
b2 = zeros(StaticArrays.SArray{Tuple{2},Float64,1,2},N)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    nd = (k-1)*Nxy+(j-1)*Nx+i
    A[(2*nd-1):2*nd, (2*nd-1):2*nd] = S[i,j,k].value[4]
    A2[nd,nd] = S[i,j,k].value[4]
    if i!=1  
        A[(2*nd-1):2*nd, (2*nd-3):(2*nd-2)]           = S[i,j,k].value[1] 
        A2[nd, nd-1] = S[i,j,k].value[1]
    end
    if j!=1  
        A[(2*nd-1):2*nd, (2*nd-2*Nx-1):(2*nd-2*Nx)]   = S[i,j,k].value[2]
        A2[nd,nd-Nx] = S[i,j,k].value[2]
    end  
    if k!=1  
        A[(2*nd-1):2*nd, (2*nd-2*Nxy-1):(2*nd-2*Nxy)] = S[i,j,k].value[3] 
        A2[nd,nd-Nxy] = S[i,j,k].value[3]
    end
    if k!=Nz 
        A[(2*nd-1):2*nd, (2*nd+2*Nxy-1):(2*nd+2*Nxy)] = S[i,j,k].value[5] 
        A2[nd,nd+Nxy] = S[i,j,k].value[5]
    end
    if j!=Ny 
        A[(2*nd-1):2*nd, (2*nd+2*Nx-1):(2*nd+2*Nx)]   = S[i,j,k].value[6] 
        A2[nd,nd+Nx] = S[i,j,k].value[6]
    end
    if i!=Nx 
        A[(2*nd-1):2*nd, (2*nd+1):(2*nd+2)]           = S[i,j,k].value[7] 
        A2[nd,nd+1] = S[i,j,k].value[7]
    end
    b[(2*nd-1):(2*nd)] = res[i,j,k]
    b2[nd] = res[i,j,k]
    if (j==1)&&(k==1) print(i) end
end    
#ab = (A\b)

1234567891011121314151617181920

In [10]:
b2copy = copy(b2)
bcopy = copy(b);

In [29]:
mybench(S, res, copyres); # Stencil-grid A_mul_B

  513.680 μs (0 allocations: 0 bytes)


In [12]:
@btime S*res; # Stencil-grid Multiplication

  511.951 μs (3 allocations: 166.47 KiB)


In [13]:
@btime A_mul_B!(1, A2, b2, 2, b2copy); #Block sparse matrix A_mul_B

  165.968 μs (0 allocations: 0 bytes)


In [14]:
@btime A*b; # Sparse Matrix-vector multiplication

  107.770 μs (2 allocations: 125.08 KiB)


In [15]:
@btime A_mul_B!(1, A, b, 2, bcopy); # Sparse Matrix-vector A_mul_B!

  105.617 μs (0 allocations: 0 bytes)


In [27]:
function mybench(aaa, xxx, yyy)
    @btime A_mul_B!(1.0, $aaa, $xxx, 2.0, $yyy);
end

mybench (generic function with 1 method)

In [23]:
function testme(afun)
    @btime $afun(2) 
end

testme (generic function with 1 method)

In [22]:
@code_warntype S*res

Variables:
  #self# <optimized out>
  S::Stencil{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7,Array{StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7},3}}
  x::Grid{StaticArrays.SArray{Tuple{2},Float64,1,2},3,7,Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}}

Body:
  begin 
      SSAValue(0) = (Core.getfield)(x::Grid{StaticArrays.SArray{Tuple{2},Float64,1,2},3,7,Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}}, :A)::Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}
      $(Expr(:inbounds, false))
      # meta: location abstractarray.jl zero 751
      SSAValue(4) = $(Expr(:invoke, MethodInstance for similar(::Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}, ::Type{StaticArrays.SArray{Tuple{2},Float64,1,2}}), :(Base.similar), SSAValue(0), StaticArrays.SArray{Tuple{2},Float64,1,2}))
      # meta: location /Users/sw2030/.julia/v0.6/StaticArrays/src/linalg.jl zero 142
      # meta: location /Users/sw2030/.julia/v0.6/StaticArrays/src/arraymath.jl zeros 3
      # meta

In [23]:
@code_warntype A*b

Variables:
  #self# <optimized out>
  A::SparseMatrixCSC{Float64,Int64}
  x::Array{Float64,1}
  TS <optimized out>
  #temp#::Int64

Body:
  begin  # line 79:
      $(Expr(:inbounds, false))
      # meta: location abstractarray.jl size 29
      unless (Base.sle_int)(1, 2)::Bool goto 7
      #temp#::Int64 = (Core.getfield)(A::SparseMatrixCSC{Float64,Int64}, :m)::Int64
      goto 9
      7: 
      #temp#::Int64 = 1
      9: 
      # meta: pop location
      $(Expr(:inbounds, :pop))
      SSAValue(1) = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Float64,1}, svec(Any, Int64), Array{Float64,1}, 0, :(#temp#), 0))
      return $(Expr(:invoke, MethodInstance for A_mul_B!(::Float64, ::SparseMatrixCSC{Float64,Int64}, ::Array{Float64,1}, ::Float64, ::Array{Float64,1}), :(Base.SparseArrays.A_mul_B!), :((Base.sitofp)(Float64, 1)::Float64), :(A), :(x), :((Base.sitofp)(Float64, 0)::Float64), SSAValue(1)))
  end::Array{Float64,1}


In [69]:
@btime foo!(testA2, testx2, testy2);

  1.524 μs (2 allocations: 480 bytes)


In [40]:
@which +(testx,testy)

+(a::StaticArrays.StaticArray, b::StaticArrays.StaticArray) in StaticArrays at /Users/sw2030/.julia/v0.6/StaticArrays/src/linalg.jl:10

In [61]:
BLAS.gemm!('N','N', 1.0, testA, testx, 1.0, testy)

LoadError: [91mMethodError: no method matching gemm!(::Char, ::Char, ::Float64, ::StaticArrays.SArray{Tuple{2,2},Float64,2,4}, ::StaticArrays.SArray{Tuple{2},Float64,1,2}, ::Float64, ::StaticArrays.SArray{Tuple{2},Float64,1,2})[0m
Closest candidates are:
  gemm!(::Char, ::Char, ::Float64, [91m::Union{Union{Base.ReshapedArray{Float64,1,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{DenseArray, SubArray{T,N,P,I,true} where I<:Tuple{Union{Base.Slice, UnitRange},Vararg{Any,N} where N} where P where N where T}, DenseArray{Float64,1}, SubArray{Float64,1,A,I,L} where L} where I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex, Int64, Range{Int64}},N} where N} where A<:Union{Base.ReshapedArray{T,N,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:Union{DenseArray, SubArray{T,N,P,I,true} where I<:Tuple{Union{Base.Slice, UnitRange},Vararg{Any,N} where N} where 

In [62]:
A_mul_B!(1.0, testA, testx, 1.0, testy)

LoadError: [91mMethodError: no method matching A_mul_B!(::Float64, ::StaticArrays.SArray{Tuple{2,2},Float64,2,4}, ::StaticArrays.SArray{Tuple{2},Float64,1,2}, ::Float64, ::StaticArrays.SArray{Tuple{2},Float64,1,2})[0m
Closest candidates are:
  A_mul_B!(::Number, [91m::DistributedArrays.DArray{T,2,A} where A where T[39m, ::AbstractArray{T,1} where T, ::Number, [91m::DistributedArrays.DArray{T,1,A} where A where T[39m) at /Users/sw2030/.julia/v0.6/DistributedArrays/src/linalg.jl:89
  A_mul_B!([91m::Union{StaticArrays.StaticArray{Tuple{#s52,#s51},T,2} where #s51 where #s52, StaticArrays.StaticArray{Tuple{#s53},T,1} where #s53} where T[39m, ::StaticArrays.StaticArray{Tuple{N,M},T,2} where T where M where N, ::StaticArrays.StaticArray{Tuple{N},T,1} where T where N) at /Users/sw2030/.julia/v0.6/StaticArrays/src/matrix_multiply.jl:45
  A_mul_B!([91m::AbstractArray{T,1} where T[39m, ::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T, ::AbstractArray{T,1} where T) at linalg/matmu

In [24]:
function foo1!(a, x, y, n, s)
    for i in 1:n
        y[i*s] = a*x[i*s]+y[i*s]
    end
end

foo1! (generic function with 1 method)

In [26]:
xx = randn(20000)
yy = randn(20000)
yy2 = randn(20000)
a = 2.0;

In [35]:
@btime foo1!(a, xx, yy, 1000, 6)

  1.882 μs (0 allocations: 0 bytes)


In [32]:
@btime foo1!(a, xx, yy, 1000, 10)

  2.500 μs (0 allocations: 0 bytes)


In [39]:
S.v[2]

StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}(([0.0 0.0; 6.95011 0.0], [0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0], [-7.48794 -1.24799e7; -43.1777 1.03349e7], [0.0 0.0; 0.00518722 0.0], [0.0 0.0; 3.3063 0.0], [0.0 0.0; 8.11233 0.0]))

In [22]:
Profile.clear()
for i in 1:100
    @profile S*res
end
Profile.print()

183 ./task.jl:335; (::IJulia.##14#17)()
 183 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  183 .../Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1}...
   183 ...rc/execute_request.jl:158; execute_request(::ZMQ.Socket, ::...
    183 .../Compat/src/Compat.jl:71; include_string(::Module, ::Strin...
     183 ./loading.jl:522; include_string(::String, ::String)
      183 ./<missing>:?; anonymous
       183 ./In[22]:3; macro expansion
        183 ./profile.jl:23; macro expansion
         183 ...new/src/Stencil.jl:143; *(::Stencil{StaticArrays.SAr...
          1   ...new/src/Stencil.jl:130; A_mul_B!(::Float64, ::Stenc...
          165 ...new/src/Stencil.jl:132; A_mul_B!(::Float64, ::Stenc...
           3 ...rrays/src/linalg.jl:10; +
            3 ...s/src/mapreduce.jl:11; map
             3 ...s/src/mapreduce.jl:20; _map
              3 .../src/mapreduce.jl:29; macro expansion
           5 .../matrix_multiply.jl:39; *
            5 ...matrix_multiply.jl:74; _mul
           

In [23]:
Profile.clear()
for i in 1:100
    @profile A*b
end
Profile.print()

1169 ./task.jl:335; (::IJulia.##14#17)()
 1169 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  1169 .../Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1}...
   1169 ...c/execute_request.jl:158; execute_request(::ZMQ.Socket, :...
    1169 ...Compat/src/Compat.jl:71; include_string(::Module, ::Stri...
     1169 ./loading.jl:522; include_string(::String, ::String)
      1168 ./<missing>:?; anonymous
       1168 ./In[23]:3; macro expansion
        1    ./profile.jl:19; macro expansion
        1167 ./profile.jl:23; macro expansion
         1164 ./linalg/matmul.jl:79; *(::SparseMatrixCSC{Float64,...
          35  ./sparse/linalg.jl:62; A_mul_B!(::Float64, ::Sparse...
           35 ./array.jl:228; fill!(::Array{Float64,1}, :...
          48  ./sparse/linalg.jl:65; A_mul_B!(::Float64, ::Sparse...
          8   ./sparse/linalg.jl:73; A_mul_B!(::Float64, ::Sparse...
          237 ./sparse/linalg.jl:74; A_mul_B!(::Float64, ::Sparse...
          645 ./sparse/linalg.jl:75; A_mul_B!(

In [18]:
::Sindx{3,7}()()[1]

LoadError: [91msyntax: invalid "::" syntax[39m

In [34]:
@btime S[1,1,1].value[1];

  131.031 ns (3 allocations: 528 bytes)


In [35]:
@btime S.v[1,1,1].value[1];

  126.191 ns (3 allocations: 528 bytes)


In [21]:
forheatmap = [[psps[t][i,j,20][1] for i in 1:Nx, j in 1:Ny] for t in 1:30];

In [25]:
S*res

30×30×30 Grid{StaticArrays.SArray{Tuple{2},Float64,1,2},3,7,Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}}:
[:, :, 1] =
 [2.34294e8, -1.94025e8]  [0.0, 0.0]  …  [0.0, 0.0]  [3.53213e7, -2.92505e7]
 [0.0, 0.0]               [0.0, 0.0]     [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]     [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]     [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]     [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]  …  [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]     [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]     [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]     [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]     [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0.0, 0.0]  …  [0.0, 0.0]  [0.0, 0.0]             
 [0.0, 0.0]               [0

In [29]:
init       = [SVector{2,Float64}([6000.0+randn(),0.2]) for i in 1:Nx, j in 1:Ny, k in 1:Nz]
testgrid   = makegrid(init, 7);
g_guess   = testgrid;
S = getstencil(model, q, g_guess, testgrid)
res = getresidual(model, q, g_guess, testgrid);

In [31]:
S*res

30×30×30 Grid{StaticArrays.SArray{Tuple{2},Float64,1,2},3,7,Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}}:
[:, :, 1] =
 [2.30708e8, -1.91055e8]  [-8.33815e7, 6.90521e7]  …  [3.85506e7, -3.19248e7]
 [2.17404e7, -1.80037e7]  [1.02807e8, -8.51394e7]     [-4.08314e6, 3.38139e6]
 [-6.09498e7, 5.0475e7]   [6.782e7, -5.61647e7]       [1.7431e6, -1.4435e6]  
 [5.57118e7, -4.61375e7]  [-8.90947e7, 7.37832e7]     [3.66068e6, -3.03151e6]
 [-1.17452e7, 9.72688e6]  [1.76125e7, -1.45856e7]     [-2.40083e6, 1.98819e6]
 [-1.04178e7, 8.62724e6]  [4.69524e6, -3.88824e6]  …  [9.97566e5, -8.2611e5] 
 [9.99622e6, -8.27818e6]  [3.02304e6, -2.50345e6]     [-9.19415e5, 7.61391e5]
 [-324817.0, 2.69005e5]   [-1.05563e7, 8.74195e6]     [69338.9, -57421.2]    
 [-5.4673e6, 4.52762e6]   [2.93322e6, -2.42907e6]     [-1.70154e5, 140909.0] 
 [6.12402e6, -5.07148e6]  [5.36133e6, -4.43986e6]     [55343.0, -45830.9]    
 [-7.63526e5, 632313.0]   [-1.13776e6, 9.42223e5]  …  [126417.0, -1.04689e5] 
 [-3.47187e7, 2.8

In [38]:
N = Nx*Ny*Nz
Nxy = Nx*Ny
A = spzeros(2N,2N)
b = zeros(2N)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    nd = (k-1)*Nxy+(j-1)*Nx+i
    A[(2*nd-1):2*nd, (2*nd-1):2*nd] = S[i,j,k].value[4]
    if i!=1  A[(2*nd-1):2*nd, (2*nd-3):(2*nd-2)]           = S[i,j,k].value[1] end
    if j!=1  A[(2*nd-1):2*nd, (2*nd-2*Nx-1):(2*nd-2*Nx)]   = S[i,j,k].value[2] end  
    if k!=1  A[(2*nd-1):2*nd, (2*nd-2*Nxy-1):(2*nd-2*Nxy)] = S[i,j,k].value[3] end
    if k!=Nz A[(2*nd-1):2*nd, (2*nd+2*Nxy-1):(2*nd+2*Nxy)] = S[i,j,k].value[5] end
    if j!=Ny A[(2*nd-1):2*nd, (2*nd+2*Nx-1):(2*nd+2*Nx)]   = S[i,j,k].value[6] end
    if i!=Nx A[(2*nd-1):2*nd, (2*nd+1):(2*nd+2)]           = S[i,j,k].value[7] end
    b[(2*nd-1):(2*nd)] = res[i,j,k]
    if (j==1)&&(k==1) print(i) end
end    

#ab = (A\b)

1234567891011121314151617181920

In [43]:
PrecondP = zeros(A)
for i in 1:size(A,1), j in 1:size(A,2)
    if abs(i-j)<3.5 PrecondP[i,j] = A[i,j] end
end
E = A-PrecondP
P_A = inv(inv(full(PrecondP))*(eye(A)-E*inv(full(PrecondP))));
P_A_inv = (inv(full(PrecondP))*eye(A)-E*inv(full(PrecondP)));

LoadError: [91mInterruptException:[39m

In [14]:
using IterativeSolvers

In [19]:
@time IterativeSolvers.gmres(A, b;log=true, restart = 100, maxiter = 100,tol=1e-8)

  0.511913 seconds (6.73 k allocations: 42.808 MiB, 0.75% gc time)


([0.540215, -1.05917e-5, 0.00565015, -7.81858e-7, 2.74079e-5, -1.18236e-8, 6.44729e-8, -6.28496e-11, 4.49943e-11, -1.08009e-13  …  3.59529e-19, -6.10257e-22, 5.66507e-15, -7.47446e-18, 1.17551e-10, -9.7459e-14, 1.51884e-6, -4.77371e-10, 0.00864133, -3.8173e-6], Not converged after 100 iterations.)

In [42]:
@time stencilgmres(S, res, 100; maxiter=100,ifprint=true ,tol=1e-8);

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100Maxiter394.448557 seconds (5.13 M allocations: 410.679 GiB, 11.60% gc time)


In [36]:
@btime Base.LinAlg.vecnorm2(res.A)

  91.453 μs (1 allocation: 16 bytes)


14027.35222122607

In [39]:
@btime A_mul_B!(-1.0, S, res, 1.0, res2);

  2.680 ms (0 allocations: 0 bytes)


In [41]:
@btime A_mul_B!(-1.0, A, b, 1.0, c);

  466.558 μs (0 allocations: 0 bytes)


In [31]:
@time IterativeSolvers.gmres(A, b; Pl=lufact!(full(P_A)),log=true,restart=100,tol=1e-8)

([0.0115504, -6.93026e-9, 1.54878e-5, -9.29268e-12, 2.23621e-8, -1.34173e-14, 4.10156e-11, -2.46094e-17, 5.79026e-14, -3.47416e-20  …  1.17908e-5, -7.0745e-12, 3.23859e-5, -1.94315e-11, 7.39067e-5, -4.4344e-11, 0.000349556, -2.09734e-10, 0.0440842, -2.64505e-8], Converged after 27 iterations.)

In [33]:
@time result_g2 = stencilgmres(S, res, 100;maxiter=100,ifprint=true, M = (t->P_A_Stencil*t),tol=1e-8);

123456789101112131415161718192021222324252627

16×16×16 Grid{StaticArrays.SArray{Tuple{2},Float64,1,2},3,7,Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}}:
[:, :, 1] =
 [0.0115504, -6.93026e-9]     …  [0.0486194, -2.91719e-8]   
 [1.54878e-5, -9.29269e-12]      [0.000250807, -1.50485e-10]
 [2.23621e-8, -1.3403e-14]       [1.18658e-6, -7.11492e-13] 
 [4.10156e-11, -2.57871e-17]     [2.55198e-9, -1.52975e-15] 
 [5.79026e-14, -1.01312e-18]     [2.24175e-12, -4.52068e-19]
 [1.19567e-16, 2.49023e-19]   …  [1.25604e-15, 2.51088e-20] 
 [2.50101e-18, 4.67675e-19]      [1.61989e-17, 1.12865e-19] 
 [3.24755e-20, 4.93713e-20]      [6.9593e-15, 2.38987e-18]  
 [8.57212e-21, 2.89745e-19]      [1.78068e-12, 1.96307e-16] 
 [3.7637e-20, 3.74101e-21]       [2.59303e-10, -4.63816e-15]
 [2.4294e-17, -3.17708e-23]   …  [2.93724e-8, -6.23282e-15] 
 [1.87968e-14, -1.11333e-20]     [2.74296e-6, -2.09077e-12] 
 [5.18497e-11, -3.11096e-17]     [9.00098e-5, -5.35543e-11] 
 [1.1075e-7, -6.64498e-14]       [0.00163789, -9.82581e-10] 
 [7.5718e-5, -4.54308

In [34]:
N = Nx*Ny*Nz
Nxy = Nx*Ny
A = spzeros(2N,2N)
b = zeros(2N)
bb = zeros(2N)
bbb = zeros(2N)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    nd = (k-1)*Nxy+(j-1)*Nx+i
    A[(2*nd-1):2*nd, (2*nd-1):2*nd] = S[i,j,k].value[4]
    if i!=1  A[(2*nd-1):2*nd, (2*nd-3):(2*nd-2)]           = S[i,j,k].value[1] end
    if j!=1  A[(2*nd-1):2*nd, (2*nd-2*Nx-1):(2*nd-2*Nx)]   = S[i,j,k].value[2] end  
    if k!=1  A[(2*nd-1):2*nd, (2*nd-2*Nxy-1):(2*nd-2*Nxy)] = S[i,j,k].value[3] end
    if k!=Nz A[(2*nd-1):2*nd, (2*nd+2*Nxy-1):(2*nd+2*Nxy)] = S[i,j,k].value[5] end
    if j!=Ny A[(2*nd-1):2*nd, (2*nd+2*Nx-1):(2*nd+2*Nx)]   = S[i,j,k].value[6] end
    if i!=Nx A[(2*nd-1):2*nd, (2*nd+1):(2*nd+2)]           = S[i,j,k].value[7] end
    b[(2*nd-1):(2*nd)] = res[i,j,k]
    bb[(2*nd-1):(2*nd)] = result_g[i,j,k]
    bbb[(2*nd-1):(2*nd)] = result_g2[i,j,k]
    if (j==1)&&(k==1) print(i) end
end    
ab = (A\b)

12345678910111213141516

8192-element Array{Float64,1}:
  0.0115504  
 -6.93026e-9 
  1.54878e-5 
 -9.29268e-12
  2.23621e-8 
 -1.34173e-14
  4.10156e-11
 -2.46094e-17
  5.79026e-14
 -3.47416e-20
  1.19564e-16
 -7.17382e-23
  2.49444e-18
  ⋮          
  7.52614e-6 
 -4.51568e-12
  1.17909e-5 
 -7.07453e-12
  3.23859e-5 
 -1.94315e-11
  7.39067e-5 
 -4.4344e-11 
  0.000349556
 -2.09734e-10
  0.0440842  
 -2.64505e-8 

In [35]:
maximum(abs.(ab-bb))

2.270991796764079

In [36]:
maximum(abs.(ab-bbb))

0.00018114996381795025

In [37]:
norm(ab-bb), norm(ab-bbb)

(2.8959908930086975, 0.00033804160815213566)

In [43]:
SS

16×16×16 Array{StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7},3}:
[:, :, 1] =
 StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}(([0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0], [-12.7868 -2.13113e7; -42.4382 1.76485e7], [0.0 0.0; 0.000281267 0.0], [0.0 0.0; 0.0186915 0.0], [0.0 0.0; 0.0623274 0.0]))        …  StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}(([0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0726078 0.0], [0.0 0.0; 0.0 0.0], [-18.6878 -3.11464e7; -62.3841 2.57931e7], [0.0 0.0; 0.00188046 0.0], [0.0 0.0; 0.0 0.0], [0.0 0.0; 0.402621 0.0]))        
 StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}(([0.0 0.0; 0.0623274 0.0], [0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0], [-11.1865 -1.86442e7; -37.219 1.54397e7], [0.0 0.0; 0.000399631 0.0], [0.0 0.0; 0.0128286 0.0], [0.0 0.0; 0.0880639 0.0]))      StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}(([0.0 0.0; 0.402621 0.0], [0.0 0.0; 0.0854413 0.0], [0.0 0.0; 0.0 0.0], [-18

In [32]:
N = Nx*Ny*Nz
Nxy = Nx*Ny
stencilarray_P = Array{StencilPoint{typeof(@SMatrix(randn(2,2))),3,7},3}(Nx,Ny,Nz)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    temp = [@SMatrix(zeros(2,2)) for xxx in 1:7]
    nd = (k-1)*Nxy+(j-1)*Nx+i
    temp[4] = P_A_inv[(2*nd-1):2*nd, (2*nd-1):2*nd]
    if i!=1  temp[1] = SMatrix{2,2}(P_A_inv[(2*nd-1):2*nd, (2*nd-3):(2*nd-2)])           end
    if j!=1  temp[2] = SMatrix{2,2}(P_A_inv[(2*nd-1):2*nd, (2*nd-2*Nx-1):(2*nd-2*Nx)])   end  
    if k!=1  temp[3] = SMatrix{2,2}(P_A_inv[(2*nd-1):2*nd, (2*nd-2*Nxy-1):(2*nd-2*Nxy)]) end
    if k!=Nz temp[5] = SMatrix{2,2}(P_A_inv[(2*nd-1):2*nd, (2*nd+2*Nxy-1):(2*nd+2*Nxy)]) end
    if j!=Ny temp[6] = SMatrix{2,2}(P_A_inv[(2*nd-1):2*nd, (2*nd+2*Nx-1):(2*nd+2*Nx)])   end
    if i!=Nx temp[7] = SMatrix{2,2}(P_A_inv[(2*nd-1):2*nd, (2*nd+1):(2*nd+2)])           end  
    stencilarray_P[i,j,k] = StencilPoint{eltype(temp),3,7}(Tuple(temp))
end  
P_A_Stencil = Stencil{typeof(stencilarray_P[1,1,1].value[1]),3,7,typeof(stencilarray_P)}(stencilarray_P);

In [29]:
stencilarray = Array{StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7},3}(Nx,Ny,Nz)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    temp = (@SMatrix(zeros(2,2)), @SMatrix(zeros(2,2)),@SMatrix(zeros(2,2)),@SMatrix([randn() 0; 0.0 randn()]),
                        @SMatrix(zeros(2,2)),@SMatrix(zeros(2,2)),@SMatrix(zeros(2,2)))
    stencilarray[i,j,k] = StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}(temp)
end
precond_M = Stencil{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7,typeof(stencilarray)}(stencilarray);

In [35]:
eltype(S.v)

StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}

In [31]:
N = Nx*Ny*Nz
Nxy = Nx*Ny
P_A = spzeros(2N,2N)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    nd = (k-1)*Nxy+(j-1)*Nx+i
    P_A[(2*nd-1):2*nd, (2*nd-1):2*nd] = precond_M[i,j,k].value[4]
    if i!=1  P_A[(2*nd-1):2*nd, (2*nd-3):(2*nd-2)]           = precond_M[i,j,k].value[1] end
    if j!=1  P_A[(2*nd-1):2*nd, (2*nd-2*Nx-1):(2*nd-2*Nx)]   = precond_M[i,j,k].value[2] end  
    if k!=1  P_A[(2*nd-1):2*nd, (2*nd-2*Nxy-1):(2*nd-2*Nxy)] = precond_M[i,j,k].value[3] end
    if k!=Nz P_A[(2*nd-1):2*nd, (2*nd+2*Nxy-1):(2*nd+2*Nxy)] = precond_M[i,j,k].value[5] end
    if j!=Ny P_A[(2*nd-1):2*nd, (2*nd+2*Nx-1):(2*nd+2*Nx)]   = precond_M[i,j,k].value[6] end
    if i!=Nx P_A[(2*nd-1):2*nd, (2*nd+1):(2*nd+2)]           = precond_M[i,j,k].value[7] end
    if (j==1)&&(k==1) print(i) end
end   

123456

In [38]:
svdvals(full(P_A))

432-element Array{Float64,1}:
 2.79961   
 2.7067    
 2.58452   
 2.32513   
 2.28027   
 2.26918   
 2.21877   
 2.21218   
 2.18611   
 2.1539    
 2.13461   
 2.08256   
 2.04268   
 ⋮         
 0.0245877 
 0.0239329 
 0.0225562 
 0.0200639 
 0.0190295 
 0.0180838 
 0.0117689 
 0.0110942 
 0.00939585
 0.00766536
 0.00698825
 0.00507583

In [20]:
using IterativeSolvers

INFO: Recompiling stale cache file /Users/sw2030/.julia/lib/v0.6/IterativeSolvers.ji for module IterativeSolvers.


In [14]:
S.v[1,1,1].value[1] = zeros(2,2)

LoadError: [91mMethodError: no method matching setindex!(::NTuple{7,StaticArrays.SArray{Tuple{2,2},Float64,2,4}}, ::Array{Float64,2}, ::Int64)[39m

In [15]:
testa = (1,2,3)

(1, 2, 3)

In [20]:
Base.setindex(testa,4.0,1)

(4.0, 2, 3)

In [24]:
Base.setindex!(S[1,1,1].value, @SMatrix(randn(2,2)),4)

LoadError: [91mMethodError: no method matching setindex!(::NTuple{7,StaticArrays.SArray{Tuple{2,2},Float64,2,4}}, ::StaticArrays.SArray{Tuple{2,2},Float64,2,4}, ::Int64)[39m

In [22]:
a = 0
for i in 1:1000000
    a += 1
end
a

1000000

In [34]:
StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}(Base.setindex(S[1,1,1].value, @SMatrix(randn(2,2)),4))

StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7}(([0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0], [0.0806897 1.99648; -0.508091 0.692843], [0.0 0.0; 0.000450028 0.0], [0.0 0.0; 0.0299064 0.0], [0.0 0.0; 0.0997239 0.0]))

In [10]:
@btime A*b;

  109.203 μs (2 allocations: 125.08 KiB)


In [28]:
AA = spzeros(StaticArrays.SArray{Tuple{2},Float64,1,2},2,2)

2×2 SparseMatrixCSC{StaticArrays.SArray{Tuple{2},Float64,1,2},Int64} with 0 stored entries

In [29]:
AA[1] = @SVector(randn(2))

2-element StaticArrays.SArray{Tuple{2},Float64,1,2}:
 -0.175136
 -0.72164 

In [30]:
AA[2], AA[3], AA[4] = @SVector(zeros(2)), @SVector(zeros(2)), @SVector(zeros(2))

([0.0, 0.0], [0.0, 0.0], [0.0, 0.0])

In [31]:
AA

2×2 SparseMatrixCSC{StaticArrays.SArray{Tuple{2},Float64,1,2},Int64} with 4 stored entries:
  [1, 1]  =  [-0.175136, -0.72164]
  [2, 1]  =  [0.0, 0.0]
  [1, 2]  =  [0.0, 0.0]
  [2, 2]  =  [0.0, 0.0]

In [23]:
sparse(AA)

2×2 SparseMatrixCSC{StaticArrays.SArray{Tuple{2},Float64,1,2},Int64} with 4 stored entries:
  [1, 1]  =  [0.597803, 0.0415162]
  [2, 1]  =  [0.0, 0.0]
  [1, 2]  =  [0.0, 0.0]
  [2, 2]  =  [0.0, 0.0]

LoadError: [91mMethodError: no method matching dropzeros(::Array{StaticArrays.SArray{Tuple{2},Float64,1,2},2})[0m
Closest candidates are:
  dropzeros([91m::SparseMatrixCSC[39m) at sparse/sparsematrix.jl:1231
  dropzeros([91m::SparseMatrixCSC[39m, [91m::Bool[39m) at sparse/sparsematrix.jl:1231
  dropzeros([91m::SparseVector[39m) at sparse/sparsevector.jl:1953
  ...[39m

In [11]:
@btime S*res;

  497.690 μs (3 allocations: 166.47 KiB)


In [21]:
@code_warntype S*res

Variables:
  #self# <optimized out>
  S::Stencil{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7,Array{StencilPoint{StaticArrays.SArray{Tuple{2,2},Float64,2,4},3,7},3}}
  x::Grid{StaticArrays.SArray{Tuple{2},Float64,1,2},3,7,Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}}

Body:
  begin 
      SSAValue(0) = (Core.getfield)(x::Grid{StaticArrays.SArray{Tuple{2},Float64,1,2},3,7,Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}}, :A)::Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}
      $(Expr(:inbounds, false))
      # meta: location abstractarray.jl zero 751
      SSAValue(4) = $(Expr(:invoke, MethodInstance for similar(::Array{StaticArrays.SArray{Tuple{2},Float64,1,2},3}, ::Type{StaticArrays.SArray{Tuple{2},Float64,1,2}}), :(Base.similar), SSAValue(0), StaticArrays.SArray{Tuple{2},Float64,1,2}))
      # meta: location /Users/sw2030/.julia/v0.6/StaticArrays/src/linalg.jl zero 142
      # meta: location /Users/sw2030/.julia/v0.6/StaticArrays/src/arraymath.jl zeros 3
      # meta

In [39]:
for i in 1:100
    @profile S*res
end

In [40]:
Profile.print()

1135 ./task.jl:335; (::IJulia.##14#17)()
 1135 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  1135 .../Compat/src/Compat.jl:385; (::Compat.#inner#17{Array{Any,1}...
   1055 ...c/execute_request.jl:158; execute_request(::ZMQ.Socket, :...
    1055 ...Compat/src/Compat.jl:71; include_string(::Module, ::Stri...
     1055 ./loading.jl:522; include_string(::String, ::String)
      1055 ./<missing>:?; anonymous
       1055 ./In[39]:2; macro expansion
        1055 ./profile.jl:23; macro expansion
         1055 ...new/src/Stencil.jl:139; *(::Stencil{StaticArrays.SA...
          23  ./multidimensional.jl:843; fill!(::Array{StaticArrays....
          2   ...s/src/mapreduce.jl:29; A_mul_B!(::Float64, ::Stenc...
          9   ...new/src/Stencil.jl:126; A_mul_B!(::Float64, ::Stenc...
           9 ./linalg/generic.jl:8; generic_scale!(::Array{Static...
            4 ./simdloop.jl:71; macro expansion
            5 ./simdloop.jl:73; macro expansion
             5 ./linalg/generic.jl:9; macro ex